# 40 - Experiment #2

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2" #Numéro GPU
print(os.environ["CUDA_VISIBLE_DEVICES"])

2


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
#!pip install --upgrade transformers datasets spacy transformers[sentencepiece] seqeval

## Initialisation
Set the BASE path.
If run on Google Colab, will also mout Google Drive to the moutpoint given below

In [4]:
import os, sys
from pathlib import Path

ENV_IS_GOOGLE_COLAB = True if 'google.colab' in str(get_ipython()) else False
os.environ["ENV_IS_GOOGLE_COLAB"] = str(ENV_IS_GOOGLE_COLAB)

if ENV_IS_GOOGLE_COLAB:
  from google.colab import drive
  mountpoint = Path("/content/drive")
  drive.mount(str(mountpoint)) # Mount gdrive to BASE
  base = mountpoint / "MyDrive/article_icdar_2023" # Adapt this to your situation
  sys.path.append(str(base)) # Add BASE to Python Path
  BASE = Path(base).resolve() # Make BASE absolute
  DATASETS =  BASE / "dataset"
else:
  BASE = Path(os.path.dirname(os.path.realpath("__file__"))).resolve() # If not on GColab, BASE will be the directory of this notebook
  DATASETS = Path('/work/stual/dataset_ICDAR')
  OUT_BASE = Path('/work/stual/res_ICDAR/method_0')

print(sys.path)
print(BASE)
print(DATASETS)
print(OUT_BASE)

['/lrde/home2/stual/stage_DAS/m0_flat_ner', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages']
/lrde/home2/stual/stage_DAS/m0_flat_ner
/work/stual/dataset_ICDAR
/work/stual/res_ICDAR/method_0


In [5]:
# CONTROLS
RUN_CAMEMBERT = False             # Set to false to skip training Camembert
RUN_CAMEMBERT_PRETRAINED = True  # Set to false to skip training Camembert pretrained

# Number of times a model will be trained & evaluated on each of the 8 trainsets.
N_RUNS = 5

In [6]:
assert RUN_CAMEMBERT != RUN_CAMEMBERT_PRETRAINED

In [7]:
if RUN_CAMEMBERT:
    MODEL = "Jean-Baptiste/camembert-ner"
    MODEL_NAME = 'camembert_ner'
    FOLDER = f"41-flat-ner-pero-ocr-{MODEL_NAME}"
    
if RUN_CAMEMBERT_PRETRAINED:
    MODEL = "HueyNemud/das22-10-camembert_pretrained"
    MODEL_NAME = 'pretrained_camembert_ner'
    FOLDER = f"42-flat-ner-pero-ocr-{MODEL_NAME}"

## Constants

In [8]:
import os
from pathlib import Path
from config import logger

# Expected datasets indexed by number of examples in the trainset
#TRAINSETS_SIZES = [47,95,190,380,760,1521,3042,6084] #To train on the 7 datasets
TRAINSETS_SIZES = [6084] #To train only on the biggest dataset

# INPUT / OUTPUT DIRS
INPUT_DIR = OUT_BASE / f"02-experiment_2_prepared_pero_ocr_dataset_{MODEL_NAME}"
METRICS_OUTPUT_DIR = OUT_BASE / "40-experiment_2_metrics"
INPUT_DIR, METRICS_OUTPUT_DIR

(PosixPath('/work/stual/res_ICDAR/method_0/02-experiment_2_prepared_pero_ocr_dataset_pretrained_camembert_ner'),
 PosixPath('/work/stual/res_ICDAR/method_0/40-experiment_2_metrics'))

## 22 - CamemBERT - Common

In [9]:
# COMMON CONSTANTS

TRAINING_CONFIG = {
    "evaluation_strategy": "steps",
    "eval_steps": 100, #Une éval toutes les 100 itérations ????
    "max_steps": 5000,# Nombre maximal d'itérations
    "learning_rate": 1e-4,#Vitesse d'apprentissage
    "per_device_train_batch_size": 16,#Batch size train
    "per_device_eval_batch_size": 16,#Batch size évaluation
    "weight_decay": 1e-5,#Méthode de dégradation des pondérations (limiter le sur apprentissage)
    "load_best_model_at_end": True,#Charger le meilleur modèle à la fin de l'entrainement
    "greater_is_better":True,
    "metric_for_best_model": "f1",#Métrique utilisée pour le choix du meilleur modèle F1-SCore
    "save_strategy": "steps",
    "save_steps": 100, # Make Early callback bug ?
    "save_total_limit": 1,
}

In [10]:
from config import logger
from datasets import load_from_disk
import json
from camembert_util import init_model, train_eval_loop, _convert_tokenizer

def train_bert(metrics_output_directory):
    # Train & evaluate loop
    for run in range(1, N_RUNS + 1):
        output_dir = metrics_output_directory / f"run_{run}"
        output_dir.mkdir(exist_ok=True)

        for trainset_size in TRAINSETS_SIZES:
            datasetdir = INPUT_DIR / f"huggingface_{trainset_size}"
            logger.info(f"Running on datasets in {datasetdir}")
            logger.info(f"Metrics will be saved in {output_dir}")
            
            model, tokenizer, training_args = init_model(MODEL, local_config,run)
            logger.info(f"{model} #{run}, will save in {output_dir}")

            train_dev_test = load_from_disk(datasetdir)
            train = train_dev_test["train"]
            dev = train_dev_test["dev"]
            test = train_dev_test["test"]
            metrics = train_eval_loop(model,         # Implicit. Must be setbefore calling train_bert()
                                      training_args, # Idem
                                      tokenizer,
                                      train,dev,test)

            # Save the dev and test metrics
            metrics_file = output_dir / f"test_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[0], o)

            metrics_file = output_dir / f"dev_{trainset_size}.json"
            with open(metrics_file, "w", encoding="utf-8") as o:
                json.dump(metrics[1], o)
                
            torch.cuda.empty_cache()

2023-01-19 15:27:25.669379: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-19 15:27:27.572513: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-19 15:27:27.572628: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-19 15:27:27.572642: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

## 41 - CamemBERT - train & eval on Pero OCR dataset

In [11]:
import time
import datetime

if RUN_CAMEMBERT:
    assert _convert_tokenizer.name_or_path == MODEL
    
    # MODEL CONSTS
    MODEL_METRICS_DIR = METRICS_OUTPUT_DIR / f"{FOLDER}"
    MODEL_METRICS_DIR.mkdir(exist_ok=True, parents=True)
    MODEL_OUTPUT_MODEL_PATH = OUT_BASE / f"tmp/{FOLDER}"
    print(MODEL_METRICS_DIR,MODEL_OUTPUT_MODEL_PATH)
    
    # Set config output dir
    local_config = TRAINING_CONFIG.copy() 
    local_config["output_dir"]=MODEL_OUTPUT_MODEL_PATH

    # Run the main loop
    h = time.time()
    train_bert(MODEL_METRICS_DIR)
    runtime = (time.time()- h)/N_RUNS
    print(f"Run-time is equal to {str(datetime.timedelta(seconds=runtime))}")
    torch.cuda.empty_cache()
else:
    print("Skipped finetuning model for IO labels")

Skipped finetuning model for IO labels


## 42 - CamemBERT pretrained - train & eval on Pero OCR dataset

In [12]:
import time
import datetime

if RUN_CAMEMBERT_PRETRAINED:
    assert _convert_tokenizer.name_or_path == MODEL
    
    # MODEL CONSTS
    MODEL_METRICS_DIR = METRICS_OUTPUT_DIR / f"{FOLDER}"
    MODEL_METRICS_DIR.mkdir(exist_ok=True, parents=True)
    MODEL_OUTPUT_MODEL_PATH = OUT_BASE / f"tmp/{FOLDER}"
    MODEL_METRICS_DIR, MODEL_OUTPUT_MODEL_PATH
    print(MODEL_METRICS_DIR,MODEL_OUTPUT_MODEL_PATH)

    # Set config output dir
    local_config = TRAINING_CONFIG.copy() 
    local_config["output_dir"]=MODEL_OUTPUT_MODEL_PATH

    # Run the main loop
    h = time.time()
    train_bert(MODEL_METRICS_DIR)
    runtime = (time.time()- h)/N_RUNS
    print(f"Run-time is equal to {str(datetime.timedelta(seconds=runtime))}")
    torch.cuda.empty_cache()
else:
    print("Skipped finetuning model for IO labels")

/work/stual/res_ICDAR/method_0/40-experiment_2_metrics/42-flat-ner-pero-ocr-pretrained_camembert_ner /work/stual/res_ICDAR/method_0/tmp/42-flat-ner-pero-ocr-pretrained_camembert_ner


Some weights of the model checkpoint at HueyNemud/das22-10-camembert_pretrained were not used when initializing CamembertForTokenClassification: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing CamembertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at HueyNemud/das22-10-camembert_pretrained and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this mo

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Per,Act,Loc,Cardinal,Ft,Titre
100,No log,0.229753,0.930147,0.942272,0.936170,0.955454,"{'precision': 0.927536231884058, 'recall': 0.9425625920471281, 'f1': 0.9349890430971513, 'number': 679}","{'precision': 0.8898916967509025, 'recall': 0.9499036608863198, 'f1': 0.9189189189189189, 'number': 519}","{'precision': 0.9363636363636364, 'recall': 0.9474375821287779, 'f1': 0.9418680600914435, 'number': 761}","{'precision': 0.9612068965517241, 'recall': 0.9867256637168141, 'f1': 0.9737991266375545, 'number': 678}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 7}","{'precision': 0.7, 'recall': 0.17073170731707318, 'f1': 0.2745098039215686, 'number': 41}"
200,No log,0.145264,0.946304,0.958287,0.952258,0.965705,"{'precision': 0.966374269005848, 'recall': 0.9734904270986745, 'f1': 0.9699192956713133, 'number': 679}","{'precision': 0.9109461966604824, 'recall': 0.9460500963391136, 'f1': 0.9281663516068053, 'number': 519}","{'precision': 0.9380645161290323, 'recall': 0.9553219448094612, 'f1': 0.9466145833333334, 'number': 761}","{'precision': 0.9766081871345029, 'recall': 0.9852507374631269, 'f1': 0.9809104258443465, 'number': 678}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 7}","{'precision': 0.7027027027027027, 'recall': 0.6341463414634146, 'f1': 0.6666666666666667, 'number': 41}"
300,No log,0.114246,0.955621,0.962384,0.958991,0.974817,"{'precision': 0.9605839416058394, 'recall': 0.9690721649484536, 'f1': 0.9648093841642228, 'number': 679}","{'precision': 0.9395085066162571, 'recall': 0.9576107899807321, 'f1': 0.9484732824427481, 'number': 519}","{'precision': 0.9530638852672751, 'recall': 0.9605781865965834, 'f1': 0.956806282722513, 'number': 761}","{'precision': 0.9809663250366032, 'recall': 0.9882005899705014, 'f1': 0.9845701689933871, 'number': 678}","{'precision': 0.3333333333333333, 'recall': 0.14285714285714285, 'f1': 0.2, 'number': 7}","{'precision': 0.7297297297297297, 'recall': 0.6585365853658537, 'f1': 0.6923076923076923, 'number': 41}"
400,No log,0.097431,0.956474,0.965736,0.961082,0.974817,"{'precision': 0.973568281938326, 'recall': 0.9764359351988218, 'f1': 0.9750000000000001, 'number': 679}","{'precision': 0.9322033898305084, 'recall': 0.953757225433526, 'f1': 0.9428571428571428, 'number': 519}","{'precision': 0.9421593830334191, 'recall': 0.9632063074901446, 'f1': 0.9525666016894087, 'number': 761}","{'precision': 0.9852941176470589, 'recall': 0.9882005899705014, 'f1': 0.9867452135493373, 'number': 678}","{'precision': 0.6666666666666666, 'recall': 0.2857142857142857, 'f1': 0.4, 'number': 7}","{'precision': 0.7894736842105263, 'recall': 0.7317073170731707, 'f1': 0.7594936708860759, 'number': 41}"
500,0.231100,0.108134,0.950862,0.965736,0.958241,0.970261,"{'precision': 0.9608127721335269, 'recall': 0.9749631811487481, 'f1': 0.9678362573099416, 'number': 679}","{'precision': 0.9292364990689013, 'recall': 0.9614643545279383, 'f1': 0.9450757575757575, 'number': 519}","{'precision': 0.9432989690721649, 'recall': 0.961892247043364, 'f1': 0.9525048796356539, 'number': 761}","{'precision': 0.9794721407624634, 'recall': 0.9852507374631269, 'f1': 0.9823529411764708, 'number': 678}","{'precision': 0.5, 'recall': 0.42857142857142855, 'f1': 0.4615384615384615, 'number': 7}","{'precision': 0.7837837837837838, 'recall': 0.7073170731707317, 'f1': 0.7435897435897435, 'number': 41}"
600,0.231100,0.091022,0.956795,0.964991,0.960875,0.975576,"{'precision': 0.9735294117647059, 'recall': 0.9749631811487481, 'f1': 0.9742457689477557, 'number': 679}","{'precision': 0.9358490566037736, 'recall': 0.9556840077071291, 'f1': 0.9456625357483317, 'number': 519}","{'precision': 0.9483204134366925, 'recall': 0.9645203679369251, 'f1': 0.9563517915309447, 'number': 761}","{'precision': 0.9809663250366032, 'recall': 0.9882005899705014, 'f1': 0.9845701689933871, 'number': 678}","{'precision': 0.75, 'recall': 0.42857142857142855, 'f1': 0.5454545454545454, 'number': 7}","{'p

The following columns in the evaluation set don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: ner_tags, tokens. If ner_tags, tokens are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 676
  Batch size = 16
/lrde/home2/stual/stage_DAS/m0_flat_ner/camembert_util.py:101: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to lo

Trainer is attempting to log a value of "{'precision': 0.3333333333333333, 'recall': 0.14285714285714285, 'f1': 0.2, 'number': 7}" of type <class 'dict'> for key "eval/FT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.7297297297297297, 'recall': 0.6585365853658537, 'f1': 0.6923076923076923, 'number': 41}" of type <class 'dict'> for key "eval/TITRE" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Saving model checkpoint to /work/stual/res_ICDAR/method_0/tmp/42-flat-ner-pero-ocr-pretrained_camembert_ner/checkpoint-300
Configuration saved in /work/stual/res_ICDAR/method_0/tmp/42-flat-ner-pero-ocr-pretrained_camembert_ner/checkpoint-300/config.json
Model weights saved in /work/stual/res_ICDAR/method_0/tmp/42-flat-ner-pero-ocr-pretrained_camembert_ner/checkpoint-300/pytorch_model.bin
tokenizer config file saved i

...

Trainer is attempting to log a value of "{'precision': 0.9523809523809523, 'recall': 0.9633911368015414, 'f1': 0.9578544061302681, 'number': 519}" of type <class 'dict'> for key "eval/ACT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9594771241830066, 'recall': 0.9645203679369251, 'f1': 0.9619921363040629, 'number': 761}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9838235294117647, 'recall': 0.9867256637168141, 'f1': 0.9852724594992636, 'number': 678}" of type <class 'dict'> for key "eval/CARDINAL" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.4, 'recall': 0.2857142857142857, 'f1': 0.333

Deleting older checkpoint [/work/stual/res_ICDAR/method_0/tmp/42-flat-ner-pero-ocr-pretrained_camembert_ner/checkpoint-1300] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: ner_tags, tokens. If ner_tags, tokens are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 676
  Batch size = 16
Trainer is attempting to log a value of "{'precision': 0.9794117647058823, 'recall': 0.9808541973490427, 'f1': 0.9801324503311258, 'number': 679}" of type <class 'dict'> for key "eval/PER" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9489603024574669, 'recall': 0.9672447013487476, 'f1': 0.9580152671755725, 'number': 519}" of type <class 'dict'> for key "eval/ACT" as 

Configuration saved in /work/stual/res_ICDAR/method_0/tmp/42-flat-ner-pero-ocr-pretrained_camembert_ner/checkpoint-1700/config.json
Model weights saved in /work/stual/res_ICDAR/method_0/tmp/42-flat-ner-pero-ocr-pretrained_camembert_ner/checkpoint-1700/pytorch_model.bin
tokenizer config file saved in /work/stual/res_ICDAR/method_0/tmp/42-flat-ner-pero-ocr-pretrained_camembert_ner/checkpoint-1700/tokenizer_config.json
Special tokens file saved in /work/stual/res_ICDAR/method_0/tmp/42-flat-ner-pero-ocr-pretrained_camembert_ner/checkpoint-1700/special_tokens_map.json
Deleting older checkpoint [/work/stual/res_ICDAR/method_0/tmp/42-flat-ner-pero-ocr-pretrained_camembert_ner/checkpoint-1600] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: ner_tags, tokens. If ner_tags, tokens are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this

Trainer is attempting to log a value of "{'precision': 0.9512338425381903, 'recall': 0.9602609727164887, 'f1': 0.9557260920897284, 'number': 1686}" of type <class 'dict'> for key "eval/PER" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8793256433007985, 'recall': 0.9058500914076782, 'f1': 0.8923908149482215, 'number': 1094}" of type <class 'dict'> for key "eval/ACT" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9484764542936288, 'recall': 0.9580302182428652, 'f1': 0.9532293986636972, 'number': 1787}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9783228750713063, 'recall': 0.9794403198172473

Run-time is equal to 0:04:37.755047
